In [ ]:
# Main pycaret
!pip install --ignore-installed pycaret

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gc
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## The train and test aggregation files are created from these two notebooks.
### [Notebook 1](https://www.kaggle.com/code/sravanneeli/train-file-pckl-creation-from-parquet-files)
### [Notebook 2](https://www.kaggle.com/code/sravanneeli/test-file-pckl-creation-from-parquet-files)

In [ ]:
np.random.seed(42)

In [ ]:
train_df = pd.read_pickle('../input/amex-data-pckl-files/train_agg.pkl', compression='gzip')
train_labels = pd.read_csv('../input/amex-default-prediction/train_labels.csv')

In [ ]:
gc.collect()

In [ ]:
cat_cols = []
for col in train_df:
    if "last" in col:
        cat_cols.append(col)

In [ ]:
while True:
    try:
        from pycaret.classification import *
        break
    except:
        pass

In [ ]:
train_labels = train_labels.sort_values(by=['customer_ID']).drop('customer_ID', axis=1)
train_df = pd.concat([train_df, train_labels], axis=1)

In [ ]:
train_df.head()

In [ ]:
# COMPETITION METRIC FROM Konstantin Yakovlev
# https://www.kaggle.com/kyakovlev
# https://www.kaggle.com/competitions/amex-default-prediction/discussion/327534
def amex_metric_mod(y_true, y_pred):

    labels     = np.transpose(np.array([y_true, y_pred]))
    labels     = labels[labels[:, 1].argsort()[::-1]]
    weights    = np.where(labels[:,0]==0, 20, 1)
    cut_vals   = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four   = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])

    gini = [0,0]
    for i in [1,0]:
        labels         = np.transpose(np.array([y_true, y_pred]))
        labels         = labels[labels[:, i].argsort()[::-1]]
        weight         = np.where(labels[:,0]==0, 20, 1)
        weight_random  = np.cumsum(weight / np.sum(weight))
        total_pos      = np.sum(labels[:, 0] *  weight)
        cum_pos_found  = np.cumsum(labels[:, 0] * weight)
        lorentz        = cum_pos_found / total_pos
        gini[i]        = np.sum((lorentz - weight_random) * weight)

    return 0.5 * (gini[1]/gini[0] + top_four)

In [ ]:
clf1 = setup(data = train_df.drop('customer_ID', axis=1),
             session_id=2286,
             fold=5,
             categorical_features=cat_cols,
             use_gpu=True,
             target = 'target',
             silent = True)

In [ ]:
add_metric("amex_customer_metric", name="Amex Metric", score_func=amex_metric_mod)

In [ ]:
del(train_df)
gc.collect()

In [ ]:
# model training
model = compare_models(include = ['lightgbm'])

In [ ]:
gc.collect()

In [ ]:
predict_model(model, raw_score=True)

In [ ]:
test_df = pd.read_pickle('../input/amex-data-pckl-files/test_agg.pkl', compression='gzip')

In [ ]:
y_pred = []
for i in range(0, test_df.shape[0], 10000):
    y_pred.extend(predict_model(model, data=test_df.iloc[i:i+10000, :],raw_score=True)['Score_1'].to_list())

In [ ]:
sub_df = pd.read_csv('../input/amex-default-prediction/sample_submission.csv')

In [ ]:
sub_df['prediction'] = y_pred

In [ ]:
sub_df.to_csv('submission.csv', index=False)